## Import Relevant Libraries

In [ ]:
import pandas as pd
import requests as r
import json
import time
import random
from dotenv import load_dotenv
from playwright.async_api import async_playwright

from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

In [10]:
!pip show dune-client

Name: dune_client
Version: 1.7.7
Summary: A simple framework for interacting with Dune Analytics official API service.
Home-page: https://github.com/duneanalytics/dune-client
Author: Benjamin H. Smith & Dune Analytics
Author-email: ben@cow.fi
License: Apache License Version 2.0
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, Deprecated, ndjson, python-dateutil, requests, types-Deprecated, types-python-dateutil, types-PyYAML, types-requests, types-setuptools
Required-by: 


## Manual Trigger of Query Run 

- Usual API query execution will expend credits very fast
- Due to credit limits of free tier, need to circumvent by manual trigger query execution on website
- Need to use "human actions" to circumvent cloudflare blocker

In [ ]:
# Secret Environment Variables
secrets = load_dotenv('secret.env')

In [115]:
# Start Playwright manually
p = await async_playwright().start()
 
# Launch the browser
browser = await p.chromium.launch(channel="chrome",headless=False)

In [150]:
await browser.close()
await p.stop()

In [116]:
# Create a new page
page = await browser.new_page()

In [136]:
# Navigate to a URL
await page.goto("https://dune.com/discover/content/trending")

<Response url='https://dune.com/discover/content/trending' request=<Request url='https://dune.com/discover/content/trending' method='GET'>>

In [137]:
# Locate navigation panel in the form of ul tag
nav_locator = page.locator("ul[class='SidePanel_panel__bZFcx']")  # Adjust the selector as needed

# Get all nav bar items within the unordered list
nav_items = nav_locator.locator("li[class='SidePanel_section__tKVhT']")

# Category items
category_items = nav_items.nth(0)

print(f"Nav panel item count: {await nav_items.count()}")

Nav panel item count: 4


In [138]:
# We will press up to 5 buttons on the nav panel randomly to simulate human action
# May lead to sign in page faster

for i in range(5):
    category_choice = random.choice([0,1,2])
    print(f"category choice: {category_choice}")
    category = category_items.locator('li')
    await category.nth(category_choice).click()
    try:
        await page.wait_for_function("window.location.href.includes('login')",timeout=10000)
        break
    except:
        pass
    sub_category_choice = random.choice([1,2])
    options = nav_items.nth(sub_category_choice)
    option_items = options.locator('li')
    option_choice = random.choice(list(range(0,await option_items.count())))
    print(f"option choice: {option_choice}")
    chosen = option_items.nth(option_choice)
    await chosen.click()
    try:
        await page.wait_for_function("window.location.href.includes('login')",timeout=10000)
        break
    except:
        pass
    print(page.url)
    

category choice: 1
option choice: 2


In [128]:
page.url

'https://dune.com/discover/content/popular'

In [ ]:
sign_in_locator = page.locator("a[class='Button_button__MJ5pb buttonThemes_button__jfRFC buttonThemes_theme-secondary-light__KAHJx Button_size-M__fDw4z']")
is_enabled = await sign_in_locator.is_enabled()
is_visible = await sign_in_locator.is_visible()
is_hidden = await sign_in_locator.is_hidden()

print(f"Enabled: {is_enabled}, Visible: {is_visible}, Hidden: {is_hidden}")

In [62]:
# Cannot use usual locator.click() method, use javascript to force click
await page.evaluate("element => element.click()", await sign_in_locator.element_handle())

In [ ]:
# Fill up login credentials
username_locator = page.locator('input[name="username"]')
await username_locator.fill(secrets['dune-username'])

password_locator = page.locator('input[name="password"]')
await password_locator.fill(secrets['dune-password'])


In [142]:
# Click login button
login_btn_locator = page.locator('button[type="submit"]')
await login_btn_locator.click()

In [143]:
# Navigate to queries page
library_btn_locator = page.locator('div[class="HeaderDesktop_headerLink__txGSd"]').filter(has_text="Library")
await library_btn_locator.click()

In [144]:
# Search for EVM query
search_locator = page.locator('input[placeholder="Search content"]')
await search_locator.fill('top trading memecoins (solana)')


In [145]:
# Click for EVM query
query_locator =  page.locator('a[class="ContentList_contentName__OAzIv"]')
await query_locator.click()

In [146]:
# Trigger EVM query run
run_btn_locator = page.locator('button[id="run-query-button"]')
await run_btn_locator.click()

## Retrieve EVM query result

In [ ]:
dune = DuneClient(secrets['dune-api-key'])
query_result = dune.get_latest_result_dataframe(4146860) 

In [127]:
query_result

,pool_address,elapsed_time,token_address,token_symbol,total_eth_vol,action,chain
0,0x21ea55ef466c2c96fa396fd98bbe7f6d667dadfc,"11 days, 0 hours, 16 minutes, 12 seconds",0x420690b6158ba4a4c9d8d6a4355308d7a54c625a,BLK,0.02093,SELL,Ethereum
1,0x7c0d6cbe332dac916944c61956b291a4045f9bfa,"10 days, 18 hours, 6 minutes, 24 seconds",0xad86b91a1d1db15a4cd34d0634bbd4ecacb5b61a,Daram,3.51692,SELL,Ethereum
2,0xf7b44d0ecb509d1295551e68141e6639be405e63,"1 days, 14 hours, 1 minutes, 12 seconds",0x79787a05d8f01d4251c8dc17cba8cda692c69e34,PUSUKE,0.15952,SELL,Ethereum
3,0xa70cc12b444b00910f888a9e3a2e527aa0026e43,"0 days, 11 hours, 31 minutes, 12 seconds",0x8802ec8c4961582b493de02500eb8e9836c0c0e3,BRETTNESS,0.16500,SELL,Ethereum
4,0xfd81996f996f8bebd762c1a913374f2788a914f8,"4 days, 15 hours, 41 minutes, 48 seconds",0x528253715ff4eb9d6923b3ff401fcb1662ac7d66,Cancy,0.10000,SELL,Ethereum
...,...,...,...,...,...,...,...
91,0x7d1817c407abde66bd1b0071b235842983ad64c0,"12 days, 12 hours, 4 minutes, 32 seconds",0xd461a534af11ef58e9f9add73129a1f45485a8dc,KEVIN,0.13055,BUY,Base
92,0x6172c110f59f414100a1b293e88653426c5491ec,"0 days, 0 hours, 41 minutes, 54 seconds",0x4efd649e811d2513b06faa18662c136dfb4bcafa,BOGUS,9.83724,BUY,Base
93,0xe47a976ecfeeac11ebf4e84a34290c019e27b6e2,"0 days, 2 hours, 0 minutes, 48 seconds",0x48b418ddfc16cf263452dc7ee844ce67e605c677,JANET,2.53355,BUY,Base
94,0x36104bb6ce1edf37076f1cf6fbabbcdf67ddff1b,"11 days, 7 hours, 58 minutes, 32 seconds",0xecaf81eb42cd30014eb44130b89bcd6d4ad98b92,CHAD,0.01780,BUY,Base


## Retrieve Token Security Information

### BASE BUY

In [136]:
base_buy = query_result[(query_result.chain == 'Base') & (query_result.action == 'BUY') & (query_result.total_eth_vol >= 0.01)].\
sort_values(by='total_eth_vol', ascending=False)

In [137]:
base_buy

,pool_address,elapsed_time,token_address,token_symbol,total_eth_vol,action,chain
92,0x6172c110f59f414100a1b293e88653426c5491ec,"0 days, 0 hours, 41 minutes, 54 seconds",0x4efd649e811d2513b06faa18662c136dfb4bcafa,BOGUS,9.83724,BUY,Base
84,0x0ad8e473052d6dde8377393541aebe0aefe97ca0,"0 days, 1 hours, 2 minutes, 26 seconds",0x36061e106bc8af88241c001fed9ba6f3db6afdb8,GOAT,9.16729,BUY,Base
95,0xc083e0b41910dd2ffa5e891bf66519540e64c20d,"0 days, 14 hours, 37 minutes, 0 seconds",0x3a79409ba2adc86fa0a91b133cf0b07cd27760a6,FUBB,7.65113,BUY,Base
93,0xe47a976ecfeeac11ebf4e84a34290c019e27b6e2,"0 days, 2 hours, 0 minutes, 48 seconds",0x48b418ddfc16cf263452dc7ee844ce67e605c677,JANET,2.53355,BUY,Base
89,0xd53236de1defcb90db8390cf2f2aaef59d2757e2,"2 days, 19 hours, 40 minutes, 4 seconds",0x72f001154a7f5c6289b43c7f4d201f36f04b3d1b,NAYM,0.85347,BUY,Base
66,0x7fbbb372df611f9c49a2cfe5ea1d31edb0e876a6,"1 days, 19 hours, 7 minutes, 34 seconds",0x1d614c16cf109fbf12cb10c89d5a60739afbff93,CROS,0.71829,BUY,Base
71,0x3a3dc4a26d1aceae12fd1026a5856f12d20658ea,"5 days, 16 hours, 32 minutes, 8 seconds",0x55cd6469f597452b5a7536e2cd98fde4c1247ee4,LUNA,0.69835,BUY,Base
81,0x3aa6dbb4a5bea36606d41c6c43fbf6803cb088c2,"2 days, 2 hours, 21 minutes, 22 seconds",0xa8bc3459ed3828a3248da2f0bb2c36320af9572f,X,0.42324,BUY,Base
72,0x939c65dbdfa3d70bfe73a50d8156247c12f88f3a,"0 days, 14 hours, 41 minutes, 54 seconds",0xfa7c6888e9fc08bcc4d29dad1513b3b6090dae29,ACT,0.36851,BUY,Base
86,0x055388b316eb969ff780a3e082b959966ec84987,"0 days, 0 hours, 45 minutes, 2 seconds",0x02a2fa07037a6b01e8aa00d6c8b04ccea9bea7c3,BabyDoge,0.16974,BUY,Base


In [171]:
base_buy_message = "📈 Trending 🔵 Base Tokens Bought in past 15 mins 📈 \n\n"

In [ ]:
async def handle_response(response):
    # Check if the response URL contains specific text
    if "api-scanner.defiyield.app" in response.url:
        data = response.json()
        if 'project' in data['data'].keys():
            issues = data['data']['project']['CoreIssues']
            security_checkers += 'De.Fi: \n' if 'De.Fi' not in security_checkers else ''
            for e in issues:
                if e['scwTitle'] and scw['issues'] != 'Low':
                    security_checkers += '- ' + e['scwDescription'] + '\n'
        elif any(key in data['data'].keys() for key in ['liquidityAnalysis', 'holderAnalysis']):
            issues = data['data']['project']['issues']
            security_checkers += 'De.Fi: \n' if 'De.Fi' not in security_checkers else ''
            for e in issues:
                if e['scwTitle'] and scw['issues'] != 'Low':
                    security_checkers += '- ' + e['scwDescription'] + '\n'
    elif 'app.quickintel.io/api/quicki/getquickiauditfull' in response.url:
        data = response.json()
        issues = data['tokenDetails']['quickiAudit']
        security_checkers += 'QuickIntel: \n' if 'QuickIntel' not in security_checkers else ''
        for k,v in issues.items():
            if 'can' in k or 'is' in k and v == True:
                security_checkers += '- ' + k + '\n'
    elif 'app.geckoterminal.com/api/' in response.url and 'token_metric_security' in response.url:
        data = response.json()
        issues = data['included'][11]['attributes']['go_plus_token_security_data'] for e in data['included'] if e
        security_checkers += 'GoPlus: \n' if 'GoPlus' not in security_checkers else ''
        for k,v in issues.items():
            if k=='is_open_source' and  v == '0':
                security_checkers += '- ' + 'not_open_source'+ '\n'
            elif v == "1":
                security_checkers += '- ' + k + '\n'

        
        

In [172]:
api_errors = []
sub_message = ''
security_checkers = ''
page.on("response", handle_response) 
# add to main message and empty sub msg and secuirty checkers for each token iteration
for token in base_buy['token_address']:
    # 1st level honeypot check by honeypot.is
    honeypot_is = json.loads(r.get(f"https://api.honeypot.is/v2/IsHoneypot?address={token}&chainID=8453").content)
    if honeypot_is['summary']['risk'] in ['high', 'unknown'] and honeypot_is['summary']['flags']:
        print(token)
        continue
    else:
        if honeypot_is['summary']['flags']:
            security_checkers += 'Honeypot.is: \n'
            for e in honeypot_is['summary']['flags']:
                print(e['description'])
                security_checkers += '- '+ e['description'] + '\n'
    
    # rest of security checks               
    # quill ai
    quill_ai  = json.loads(r.get(f"https://check-api.quillai.network/api/v1/tokens/honeypot/information/{token}?chainId=8453"))
    if quill_ai['formatted_information']['isPairHoneypot'] == 1:
        security_checkers += f"QuillAI: \n- {quill_ai['formatted_information']['honeypotReason']}\n"
    
    # de.fi
    page.goto(f"https://de.fi/scanner/contract/{token}?chainId=8453")
    
    # quick intel
    page.goto(f"https://app.quickintel.io/scanner?type=token&chain=base&contractAddress={token}")

    # go-plus and gecko terminal stats
    pool_address = base_buy[base_buy['token_address']==token]['pool_address']
    page.goto(f"https://www.geckoterminal.com/base/pools/{pool_address}")

    
    
        
    

0x3a79409ba2adc86fa0a91b133cf0b07cd27760a6
Some of the users' wallets were siphoned.
0xa8bc3459ed3828a3248da2f0bb2c36320af9572f
0xfa7c6888e9fc08bcc4d29dad1513b3b6090dae29
0x02a2fa07037a6b01e8aa00d6c8b04ccea9bea7c3
0x62bd98d9b13299863aa7cc949293f030f49f207e
0xf1afeb7eaa9a4a16b948cf6e58802f6af84dfb34
0x0af8579bad49c396d5f5e1d7a1f6b018b039b7c4
The taxes on the token are extremely high, making it an effective honeypot. Actual rate varies, but generally >50%.
A very high amount of users cannot sell their tokens.
The source code is not available, allowing for hidden functionality.


In [141]:
https://api.honeypot.is/v2/IsHoneypot?address=0x3a79409ba2adc86fa0a91b133cf0b07cd27760a6&pair=0xc083e0b41910dd2ffa5e891bf66519540e64c20d&chainID=8453

SyntaxError: invalid syntax (3751798058.py, line 1)

In [73]:
# Close the browser
await browser.close()

# Stop Playwright
await p.stop()

In [67]:
# # Locate the shadow host element
# shadow_host = page.locator('#rXOa8 #shadow-root')  # Replace with your actual selector

# # Use JavaScript to access the closed shadow root
# shadow_content = await page.evaluate('''(element) => {
#     const shadowRoot = element.shadowRoot; // Access the shadow root
#     return shadowRoot ? shadowRoot.innerText : null; // Get inner text if available
# }''', shadow_host)

# print(shadow_content)  # This will print the content inside the closed shadow root



In [68]:
# shadow_host_locator = page.locator('#cf-chl-widget-rilsg_legacy_response')  # Adjust this selector
# await shadow_host_locator.wait_for(state='visible')

# # Click on an element inside the closed shadow root using evaluate
# await page.evaluate('''(host) => {
#     const shadowRoot = host.shadowRoot; // Accessing shadow root
#     const checkbox = shadowRoot.querySelector('input[type="checkbox"]'); // Adjust selector
#     if (checkbox) {
#         checkbox.click(); // Simulate click on checkbox
#     }
# }''', shadow_host_locator)

## Retrieve Solana

In [ ]:
#time.sleep(120) # Buffer time for query to finish running
await page.wait_for_function(f'document.querySelector("button[id=\'run-query-button\']").innerText === "Run"',timeout=120000)
print("Run Completed!")
dune = DuneClient(secrets['dune-api-key'])
query_result = dune.get_latest_result_dataframe(4171165)


Run Completed!


In [149]:
query_result

,pool_address,elapsed_time,token_address,symbol,name,total_sol_vol,action,chain
0,F31o6Z3Y7uTL8djqkMjxhqd98cxhVdL1y5CEaAc7LyQK,"0 days, 0 hours, 6 minutes, 32 seconds",BRufcetuoNQYceUpn5UTGsdXwMozpJ1hWcLfny1Wba6S,<nil>,<nil>,7.65946,BUY,Solana
1,88njUWB6K87vN6SwJvQd949Wy2Zqq3nGZmRi7NcJXVyj,"0 days, 0 hours, 7 minutes, 41 seconds",wCoBU7THmooP48ikGLHxZWmsksKkubp4vfmZbBRpump,<nil>,<nil>,101.49942,BUY,Solana
2,8FNZRJpNKCi5kegkcngEvf5DBzkx5WQj8LZYU2cKPHxi,"0 days, 0 hours, 4 minutes, 56 seconds",AS6JRj3MFZ5Qmhd2WemDEnT63u9gcdQYbZcc2v5Mpump,Rats,Democrats,52.72879,BUY,Solana
3,2afXhEAWZAHiEr6oDQxcgLuSYk6xE6CFMtP2mbxf3YLW,"0 days, 0 hours, 13 minutes, 40 seconds",8U1gw3xRE1DKknJMe3ZbZ2jRs11KSwtBZ7casvD7pump,<nil>,<nil>,122.85387,BUY,Solana
4,CAts1cHSDGiMut4cxHJXDLpLgbKEbihPdQikQ1Kre15i,"0 days, 0 hours, 11 minutes, 22 seconds",5dW7BYLWgteYKkdfEm6nMoHAQF13ENy64u2casNUpump,<nil>,<nil>,179.67599,BUY,Solana
5,C19UGYsZobYaUeRFkKH2mKKoc2Aet4hDuduktn4eVrPJ,"0 days, 0 hours, 10 minutes, 14 seconds",HfYRhjnMCnY5RjaQFnzxiitT4o8yDq3gj9GFb5Hxy5P3,DRKPONKE,DARK PONKE,187.39597,BUY,Solana
6,DjPXxzdrwMJBNc1p9d2akyJ21f38pgDz9uLKixKaqVMA,"0 days, 0 hours, 10 minutes, 7 seconds",G3MeKadE73Q8dQkQAXWdD2i2BGzq6bSSJehPdH3Jpump,<nil>,<nil>,637.57659,BUY,Solana
7,A9ceNn5ujfqjfyNPjYq1L7F3qGaCPmSH1Ju4y4Sc1F1i,"0 days, 0 hours, 6 minutes, 42 seconds",HiUnDDDA16CeNkgPH2j61NK3cXeutCiprdVJXGxpump,<nil>,<nil>,484.80495,BUY,Solana
8,H4qbNo7NkNLeUrqkbq4emZxbr2AwEqea7CGZ3WKGQSYM,"0 days, 0 hours, 10 minutes, 1 seconds",3fXQZrZ4dcpjKESh1wLjNVTgjhp23azUZRsZxtGyVxQ1,<nil>,<nil>,381.58425,BUY,Solana


In [19]:
query_result['symbol'][0]

'<nil>'

In [25]:
r.get('https://solsniffer.com/api/v1/sniffer/token/Eo16JxhZLqfNdwuttsmWJaWAPLZ3xDpUPcuUEhgdpump').json()['tokenData'].keys()

dict_keys(['indicatorData', 'tokenOverview', 'address', 'deployTime', 'externals', 'liquidityList', 'marketCap', 'ownersList', 'score', 'tokenImg', 'tokenName', 'tokenSymbol', 'updatedAt', 'auditRisk'])

In [ ]:
solana_message = "📈  New Potential 🟣 SOL 🟣 Tokens to APE!!!  📈 \n\n"


for _, row in query_result.iterrows():
    # Initialise empty placeholders
    token_name = None
    token_symbol = None
    token_address = None
    pool_created = None
    top10_holder = 0.0
    holder_cnt = None
    current_change = None
    mkt_cap = None
    volume = None
    gecko_url = None
    swap_url = None
    
    # Check for security, if fail skip token
    try:
        await page.goto(f"https://solsniffer.com/api/v1/sniffer/token/{row['token_address']}")
        await page.wait_for_load_state('networkidle')
        solsniffer = await page.evaluate("document.body.innerText") 
        solsniffer = json.loads(solsniffer)['tokenData']
    except: # Remedy if rate limit is hit, else error pop up again if api changes
        time.sleep(5)
        await page.goto(f"https://solsniffer.com/api/v1/sniffer/token/{row['token_address']}")
        await page.wait_for_load_state('networkidle')
        solsniffer = await page.evaluate("document.body.innerText") 
        solsniffer = json.loads(solsniffer)['tokenData']
        
    audit = solsniffer['auditRisk']
    if audit['mintDisabled'] and audit['freezeDisabled'] and audit['lpBurned']:
        if row['symbol'] == '<nil>':
            row['symbol'] = solsniffer['tokenSymbol']
            row['name'] = solsniffer['tokenName']
        token_name = row['name']
        token_symbol = row['symbol']
        token_address = row['token_address']
        pool_created = row['elapsed_time']
        for e in solsniffer['ownersList'][1:11]: # exclude pool %
            top10_holder += float(e['percentage'])       
    else:
        continue
    
    try:
        await page.goto(f"https://app.geckoterminal.com/api/p1/solana/pools/{row['pool_address']}")
        await page.wait_for_load_state('networkidle')
        gecko_terminal = await page.evaluate("document.body.innerText")
        gecko_terminal = json.loads(gecko_terminal)['data']['attributes']
    except: # Remedy if rate limit is hit, else error pop up again if api changes
        time.sleep(5)
        await page.goto(f"https://app.geckoterminal.com/api/p1/solana/pools/{row['pool_address']}")
        await page.wait_for_load_state('networkidle')
        gecko_terminal = await page.evaluate("document.body.innerText")
        gecko_terminal = json.loads(gecko_terminal)['data']['attributes']
    
    # Check for negative roi, if true, reject as token already dumped
    if '-' in gecko_terminal['price_percent_change']:
        continue
    
    holder_cnt = gecko_terminal['gt_score_details']['holders']
    current_change = gecko_terminal['price_percent_change']
    
    mkt_cap = round(float(gecko_terminal['fully_diluted_valuation']),2)
    if mkt_cap >= 1000000:
        mkt_cap = f"{mkt_cap / 1000000.0:.2f}M"  # Convert to millions
    elif mkt_cap >= 1000:
        mkt_cap = f"{mkt_cap / 1000.0:.2f}K"      # Convert to thousands

    volume = round(float(gecko_terminal['from_volume_in_usd']),2)
    if volume >= 1000000:
        volume = f"{volume / 1000000.0:.2f}M"  # Convert to millions
    elif volume >= 1000:
        volume = f"{volume / 1000.0:.2f}K"      # Convert to thousands

    # Add to main message
    solana_message += (f"👉 {token_name} - ${token_symbol}\
    \n 📌 Address: {token_address}\
    \n 🕒 Pool Created: {pool_created}\
    \n 👑 Top 10 Holder %: {str(top10_holder)}%\
    \n 💎 Holder Count: {holder_cnt}\
    \n 📈 Price Change: {current_change}\
    \n 🏛️ Market Cap: ${mkt_cap}\
    \n 📊 Volume: ${volume}\
    \n 🦎 GeckoTerminal Link: https://www.geckoterminal.com/solana/pools/{row['pool_address']}\
    \n 🔄 Raydium Link: {gecko_terminal['swap_url']} \n\n")

In [110]:
print(solana_message)

📈  New Potential 🟣 SOL 🟣 Tokens to APE!!! 📈 


